In [81]:
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog

In [82]:
class Task:
    def __init__(self, index, name, prereq_numbers):
        self.index = index
        self.name = name
        self.prereq_numbers = prereq_numbers
        
    def __str__(self):
        return self.name
    
    def numbers_to_tasks(self, tasks):
        self.prereq_tasks = []
        for number in self.prereq_numbers:
            self.prereq_tasks.append(tasks[number])


In [83]:
class PoSorter:
    def __init__(self):
        self.tasks: list[Task] = []
        
    def topo_sort(self):
        # Initialise the tasks
        for task in self.tasks:
            task.followers = []
        # Give each task a followers list that holds references to the tasks that
        # must follow it. i.e., this task is a pre req for tasks in the followers list.
        for task in self.tasks:
            for prereq_task in task.prereq_tasks:
                prereq_task.followers.append(task)
            task.prereq_count = len(task.prereq_tasks)
        
        ready_tasks = []
        
        # Add tasks with no prerequisites to ready_tasks
        for task in self.tasks:
            if task.prereq_count == 0:
                ready_tasks.append(task)

        self.sorted_tasks = []
        
        while ready_tasks:
            task = ready_tasks.pop(0)
            self.sorted_tasks.append(task)
            
            # Loop through task's followers
            for follower in task.followers:
                # Decrement the follower's prereq_count
                follower.prereq_count -= 1
                # if the follower's prereq_count is now 0, add it to the ready_task
                if follower.prereq_count == 0:
                    ready_tasks.append(follower)
    
    def verify_sort(self):
        tasks_successfully_sorted = 0
        for task in self.sorted_tasks:
            if all(i < self.sorted_tasks.index(task) for i in [self.sorted_tasks.index(p) for p in task.prereq_tasks]):
                tasks_successfully_sorted += 1
            else:
                return f"Task {task} should not come before any of its pre-requisite tasks."
        return f"Successfully sorted {tasks_successfully_sorted} out of {len(self.tasks)} tasks."
    
    @staticmethod
    def read_task(file_handle) -> Task:
        import re
        while task_line := file_handle.readline():
            pattern = re.compile('^\s*([\d]+),([^,]+),\s*\[(.*)\]\s*$')
            matcher = pattern.match(task_line)
            if matcher:
                index = matcher.group(1).strip()
                task_name = matcher.group(2).strip()
                prereq_numbers = matcher.group(3).strip().split(',')
                prereq_numbers = [int(n) for n in prereq_numbers if n]
                return Task(index, task_name, prereq_numbers)
    
    def load_po_file(self, filename):
        self.tasks = []
        with open(filename) as file_handle:
            while task := self.read_task(file_handle):
                self.tasks.append(task)
                
        for task in self.tasks:
            task.numbers_to_tasks(self.tasks)


In [84]:
class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.sorter = PoSorter()
        
        # Make the main interface
        self.window = tk.Tk()
        self.window.title('topological_sorting')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry('400x300')
        
        # Build the menu.
        self.menubar = tk.Menu(self.window)
        self.menu_file = tk.Menu(self.window, tearoff=False)
        self.menu_file.add_command(label='Open...', command=self.open_po, accelerator='Ctrl+O')
        self.menu_file.add_separator()
        self.menu_file.add_command(label='Exit', command=self.kill_callback)
        self.menubar.add_cascade(label='File', menu=self.menu_file)
        self.window.config(menu=self.menubar)
        
        # Build the item lists.
        frame = tk.Frame(self.window)
        frame.pack(padx=10, pady=(0, 10), side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        frame.columnconfigure(1, weight=1)
        frame.columnconfigure(3, weight=1)
        frame.rowconfigure(1, weight=1)
        
        # Unsorted list.
        inner_frame = tk.Frame(frame)
        inner_frame.grid(row=1, column=1, padx=3, pady=3, sticky='nsew')
        inner_frame.columnconfigure(1, weight=1)
        inner_frame.rowconfigure(1, weight=1)
        self.unordered_list = tk.Listbox(inner_frame)
        self.unordered_list.grid(row=1, column=1, sticky='nsew')
        scrollbar = tk.Scrollbar(inner_frame)
        scrollbar.grid(row=1, column=2, sticky='nse')
        self.unordered_list.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.unordered_list.yview)
        
        # Sort button.
        sort_button = tk.Button(frame, text='Sort', width=6, command=self.sort)
        sort_button.grid(row=1, column=2, padx=3)
        
        # Sorted list.
        inner_frame = tk.Frame(frame)
        inner_frame.grid(row=1, column=3, padx=3, pady=3, sticky='nsew')
        inner_frame.columnconfigure(1, weight=1)
        inner_frame.rowconfigure(1, weight=1)
        self.ordered_list = tk.Listbox(inner_frame)
        self.ordered_list.grid(row=1, column=1, sticky='nsew')
        scrollbar = tk.Scrollbar(inner_frame)
        scrollbar.grid(row=1, column=2, sticky='nse')
        self.ordered_list.config(yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.ordered_list.yview)
        
        self.window.bind('<Control-o>', self.ctrl_o_pressed)
        
        # Display the window.
        self.window.focus_force()
        self.window.mainloop()
        
    def kill_callback(self):
        self.window.destroy()
        
    def ctrl_o_pressed(self, event):
        self.open_po()
    def open_po(self):
        file_types = [('Partial Ordering', '*.po')]
        filename = filedialog.askopenfilename(
            defaultextension='.po',
            filetypes=file_types,
            initialdir='.',
            title='Open Partial Ordering'
        )
        if not filename: return
        
        self.unordered_list.delete(0, 'end')
        self.ordered_list.delete(0, 'end')
        self.sorter.load_po_file(filename)
        self.unordered_list.insert('end', *self.sorter.tasks)
        
    def sort(self):
        self.sorter.topo_sort()
        self.ordered_list.insert('end', *self.sorter.sorted_tasks)
        messagebox.showinfo('Sort Result', self.sorter.verify_sort())


In [85]:
App()